# importing all libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten, TimeDistributed, LSTM, Input, BatchNormalization, Conv2D, MaxPooling2D, Reshape, Conv1D, GlobalAveragePooling1D, MaxPooling1D, Lambda
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy
from keras.losses import categorical_crossentropy
import tensorflow_hub as hub
from PIL import Image
import gzip
from keras.preprocessing.sequence import pad_sequences
#import spacy
import h5py
import os
import cv2
import pickle
import re
import shutil
import glob
import gzip
%matplotlib inline

In [2]:
from tqdm import tqdm

In [3]:
import time

# creating path variables

In [4]:
path_all_datasets = "D:\\imp docs\\sem4\dn\\phoenix t\\PHOENIX-2014-T-release-v3\\PHOENIX-2014-T\\program files"
path_annotation_gzip ="D:\\imp docs\\sem4\dn\\phoenix t\\PHOENIX-2014-T-release-v3\\PHOENIX-2014-T\\annotations\\gzip annotations"

In [5]:
train_annot = "\\phoenix14t.pami0.train.annotations_only.gzip"
dev_annot = "\\phoenix14t.pami0.dev.annotations_only.gzip"
test_annot = "\\phoenix14t.pami0.test.annotations_only.gzip"

In [6]:
img_embedding = "\\emb_comp_i3d_train_zero.h5"
vocab_unique = "\\unique_vocab.pickle"
padded_gloss = "\\padded_gloss.h5"
gloss_vector_dataframe = "gloss_vector_dataframe.gzip"

In [7]:
dev_img_embedding = "\\emb_comp_i3d_dev_zero.h5"
test_img_embedding = "\\emb_comp_i3d_test_zero.h5"

# loading annotations in a dataframes

In [8]:
with gzip.open(path_annotation_gzip+train_annot,'rb') as f:
    annotation_gloss = pickle.load(f)

In [9]:
with gzip.open(path_annotation_gzip+dev_annot,'rb') as f:
    annotation_dev = pickle.load(f)

In [10]:
with gzip.open(path_annotation_gzip+test_annot,'rb')as f :
    annotation_test = pickle.load(f)

In [11]:
annotation_gloss = pd.DataFrame(annotation_gloss)

In [12]:
annotation_gloss_dev = pd.DataFrame(annotation_dev)



In [13]:
annotation_gloss_test = pd.DataFrame(annotation_test)

In [14]:
annotation_gloss.head()

,name,signer,gloss,text
0,train/11August_2010_Wednesday_tagesschau-1,Signer08,JETZT WETTER MORGEN DONNERSTAG ZWOELF FEBRUAR,und nun die wettervorhersage für morgen donner...
1,train/11August_2010_Wednesday_tagesschau-4,Signer08,ORT REGEN DURCH REGEN KOENNEN UEBERSCHWEMMUNG ...,mancherorts regnet es auch länger und ergiebig...
2,train/11August_2010_Wednesday_tagesschau-5,Signer08,NORDWEST HEUTE NACHT TROCKEN BLEIBEN SUEDWEST ...,im nordwesten bleibt es heute nacht meist troc...
3,train/11August_2010_Wednesday_tagesschau-6,Signer08,TAGSUEBER OFT REGEN GEWITTER KOENNEN MANCHMAL ...,auch am tag gibt es verbreitet zum teil kräfti...
4,train/11August_2010_Wednesday_tagesschau-7,Signer08,WOLKE LOCH SPEZIELL NORDWEST,größere wolkenlücken finden sich vor allem im ...


In [15]:
annotation_gloss_dev.head()

,name,signer,gloss,text
0,dev/11August_2010_Wednesday_tagesschau-2,Signer08,DRUCK TIEF KOMMEN,tiefer luftdruck bestimmt in den nächsten tage...
1,dev/11August_2010_Wednesday_tagesschau-3,Signer08,ES-BEDEUTET VIEL WOLKE UND KOENNEN REGEN GEWIT...,das bedeutet viele wolken und immer wieder zum...
2,dev/11August_2010_Wednesday_tagesschau-8,Signer08,WIND MAESSIG SCHWACH REGION WENN GEWITTER WIND...,meist weht nur ein schwacher wind aus untersch...
3,dev/25October_2010_Monday_tagesschau-22,Signer01,MITTWOCH REGEN KOENNEN NORDWEST WAHRSCHEINLICH...,am mittwoch hier und da nieselregen in der nor...
4,dev/05May_2011_Thursday_tagesschau-25,Signer08,JETZT WETTER WIE-AUSSEHEN MORGEN FREITAG SECHS...,und nun die wettervorhersage für morgen freita...


In [16]:
annotation_gloss_test.head()

,name,signer,gloss,text
0,test/25October_2010_Monday_tagesschau-17,Signer01,REGEN SCHNEE REGION VERSCHWINDEN NORD REGEN KO...,regen und schnee lassen an den alpen in der na...
1,test/25October_2010_Monday_tagesschau-24,Signer01,DONNERSTAG NORDWEST REGEN REGION SONNE WOLKE W...,am donnerstag regen in der nordhälfte in der s...
2,test/15December_2010_Wednesday_tagesschau-37,Signer05,KRAEFTIG AB MORGEN FRUEH MEISTENS SCHNEE SCHNE...,vom nordmeer zieht ein kräftiges tief heran un...
3,test/10March_2011_Thursday_heute-58,Signer01,WOCHENENDE SONNE SAMSTAG SCHOEN TEMPERATUR BIS...,sonnig geht es auch ins wochenende samstag ein...
4,test/14August_2009_Friday_tagesschau-62,Signer05,DEUTSCH LAND MORGEN HOCH DRUCK KOMMEN WOLKE AU...,deutschland liegt morgen unter hochdruckeinflu...


In [17]:
# we can use values attribute to directly create a list of all sentences of a particular column
#annotation_gloss_test["text"].values

# preprocessing the vocabulary

In [18]:
for i in range(len(annotation_gloss)):
    annotation_gloss.iloc[i,2] = 'startseq '+annotation_gloss.iloc[i,2].lower() + ' endseq'
    annotation_gloss.iloc[i,3] = 'startseq '+annotation_gloss.iloc[i,3].lower() + ' endseq'
    annotation_gloss.iloc[i,3] = annotation_gloss.iloc[i,3].replace(' .','')
    #annotation_gloss.iloc[i,3] =annotation_gloss.iloc[i,3][-2].replace(' ','')
 

In [19]:
for i in range(len(annotation_gloss_dev)):
    annotation_gloss_dev.iloc[i,2] = 'startseq '+annotation_gloss_dev.iloc[i,2].lower() + ' endseq'
    annotation_gloss_dev.iloc[i,3] = 'startseq '+annotation_gloss_dev.iloc[i,3].lower() + ' endseq'
    annotation_gloss_dev.iloc[i,3] = annotation_gloss_dev.iloc[i,3].replace(' .','')

In [20]:
for i in range(len(annotation_gloss_test)):
    annotation_gloss_test.iloc[i,2] = 'startseq '+annotation_gloss_test.iloc[i,2].lower() + ' endseq'
    annotation_gloss_test.iloc[i,3] = 'startseq '+annotation_gloss_test.iloc[i,3].lower() + ' endseq'
    annotation_gloss_test.iloc[i,3] = annotation_gloss_test.iloc[i,3].replace(' .','')

# creating a vocabulary of texts

In [21]:
vocabulary = []
for txt in annotation_gloss.text.values:
    vocabulary.extend(txt.split())
for txt in annotation_gloss.gloss.values:
    vocabulary.extend(txt.split())
    
for txt in annotation_gloss_dev.text.values:
    vocabulary.extend(txt.split())
for txt in annotation_gloss_dev.gloss.values:
    vocabulary.extend(txt.split())
    
for txt in annotation_gloss_test.text.values:
    vocabulary.extend(txt.split())
for txt in annotation_gloss_test.gloss.values:
    vocabulary.extend(txt.split())
print('Vocabulary Size: %d' % len(set(vocabulary)))

Vocabulary Size: 3606


# reading images embedding for train test and dev

In [22]:
#reading images feature vectors
with h5py.File(path_all_datasets+img_embedding,'r')as ab:
    ind = ab.keys()
    print(ind)
    data_train=ab['x_1'][()]

<KeysViewHDF5 ['x_1']>


In [23]:
#reading images feature vectors
with h5py.File(path_all_datasets+dev_img_embedding,'r')as ab:
    ind = ab.keys()
    print(ind)
    data_dev=ab['x_1'][()]

<KeysViewHDF5 ['x_1']>


In [24]:
temp_path_dev = "D:\\imp docs\\sem4\\dn\\phoenix t\\PHOENIX-2014-T-release-v3\\PHOENIX-2014-T\\program files"

In [25]:
data_dev.shape

(519, 512)

In [26]:
#data_dev[0]

In [27]:
 #data_dev_temp[0]

In [28]:
#reading images feature vectors
with h5py.File(path_all_datasets+test_img_embedding,'r')as ab:
    ind = ab.keys()
    print(ind)
    data_test=ab['x_1'][()]

<KeysViewHDF5 ['x_1']>


In [29]:
data_train.shape

(7096, 512)

In [30]:
data_dev.shape

(519, 512)

In [31]:
data_test.shape

(642, 512)

# creating better data representation

In [32]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

In [33]:
import tensorflow.python.ops.numpy_ops.np_config as np_config


In [34]:
np_config.enable_numpy_behavior()

In [35]:
#pip install --upgrade tensorflow-lattice

In [36]:
import tensorflow_lattice.layers as tfl

In [106]:
data_train2= data_train.copy()
data_dev2 = data_dev.copy()
data_test2 = data_test.copy()

In [107]:
data_train2.shape

(7096, 512)

In [108]:
class FeatureEmbedder(nn.Module):
    
    def __init__(self, d_feat, d_model):
        super(FeatureEmbedder, self).__init__()
        self.d_model = d_model
        self.embedder = nn.Linear(d_feat, d_model)
        
    def forward(self, x): # x - tokens (B, seq_len, d_feat)
        x = self.embedder(x)
        x = x * np.sqrt(self.d_model)
        
        return x # (B, seq_len, d_model)

In [109]:
data_train2 = torch.from_numpy(data_train2)

In [110]:
data_train2 = data_train2.to(torch.float)

In [111]:
data_train2 = data_train2.reshape(7096,512,1)

In [112]:
features = FeatureEmbedder(1,2)

In [113]:
temp_data = features(data_train2)

In [130]:
train_temp = temp_data.detach().numpy()

In [131]:
temp_data = train_temp.reshape(7096,2,512)

In [132]:
temp_data.shape

(7096, 2, 512)

In [133]:
temp_data[0]

array([[ 0.3297323 , -0.8034032 ,  0.2880745 , ..., -0.7847104 ,
         0.32792273, -0.8001906 ],
       [ 0.32038182, -0.78680277,  0.28833985, ..., -0.7749215 ,
         0.3103601 , -0.7690106 ]], dtype=float32)

# creating tokenizer dataset


In [48]:
gloss_token = []
text_token = []
for i in range(len(annotation_gloss)):
    gloss_token.append(annotation_gloss.iloc[i,2])
    text_token.append(annotation_gloss.iloc[i,3])

In [49]:
gloss_token_dev = []
text_token_dev = []
for i in range(len(annotation_gloss_dev)):
    gloss_token_dev.append(annotation_gloss_dev.iloc[i,2])
    text_token_dev.append(annotation_gloss_dev.iloc[i,3])

In [50]:
gloss_token_test = []
text_token_test = []
for i in range(len(annotation_gloss_test)):
    gloss_token_test.append(annotation_gloss_test.iloc[i,2])
    text_token_test.append(annotation_gloss_test.iloc[i,3])

# creating tokenizer

In [116]:
def create_tokenizer(vocabulary):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=3606,
                                                 oov_token="<unk>",
                                                 filters='!"#$%&()*+.,-/:;=?@[\]^_`{|}~ ')
    tokenizer.fit_on_texts(vocabulary)
    tokenizer.word_index['<pad>'] = 0
    tokenizer.index_word[0] = '<pad>'
    return tokenizer

In [117]:
tokenizer = create_tokenizer(vocabulary)

In [118]:
vocab_size = len(tokenizer.word_index)+1
print("vocalb size",vocab_size)

vocalb size 3436


In [119]:
tokenizer.index_word[1]

'<unk>'

In [120]:
def encode_sequences(tokenizer, length, lines):
    # integer encode sequences
    X = tokenizer.texts_to_sequences(lines)
    # pad sequences with 0 values
    X = pad_sequences(X, maxlen=length, padding='post')
    return X

# tokenizing train,test,dev glosses

In [121]:
train_y = encode_sequences(tokenizer,35,gloss_token)
dev_y = encode_sequences(tokenizer,35,gloss_token_dev)
test_y = encode_sequences(tokenizer,35,gloss_token_test)

In [122]:
train_y[0]

array([  2,  72,  31,   9,  70, 262, 327,   3,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0])

# hyperparameters

In [123]:
BATCH_SIZE = 64
BUFFER_SIZE = 1000
num_steps = 7096 // BATCH_SIZE

In [139]:
train_temp = temp_data.detach().numpy()

AttributeError: 'numpy.ndarray' object has no attribute 'detach'

In [140]:
data_train_temp = tf.cast(temp_data,dtype = tf.float32)

In [141]:
#data_train = tf.cast(data_train,dtype = tf.float32)

In [142]:
def map_func(img_name, cap):
   
   return img_name, cap



In [143]:
dataset = tf.data.Dataset.from_tensor_slices((data_train_temp, train_y))
dataset = dataset.map(lambda item1, item2: tf.numpy_function(map_func, [item1, item2], [tf.float32, tf.int32]),num_parallel_calls=tf.data.experimental.AUTOTUNE)
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [144]:
# dataset = tf.data.Dataset.from_tensor_slices((data_train, train_y))
# dataset = dataset.map(lambda item1, item2: tf.numpy_function(map_func, [item1, item2], [tf.float32, tf.int32]),num_parallel_calls=tf.data.experimental.AUTOTUNE)
# dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
# dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [145]:
for (batch,(img,tar)) in enumerate(dataset.as_numpy_iterator()):
    print(batch)
    print(img)
    print("********")
    print(tar)
    

0
[[[ 0.32864204 -0.8014676   0.28894752 ... -0.78571177  0.3250265
   -0.7950488 ]
  [ 0.321076   -0.7880352   0.2903987  ... -0.7805501   0.30859634
   -0.7658794 ]]

 [[ 0.328864   -0.80186164  0.28908443 ... -0.7893655   0.32644373
   -0.7975648 ]
  [ 0.32047644 -0.78697073  0.2890826  ... -0.77692026  0.3101881
   -0.7687053 ]]

 [[ 0.33263218 -0.80855155  0.2864829  ... -0.791522    0.32784596
   -0.8000543 ]
  [ 0.31945872 -0.78516394  0.29029298 ... -0.77515835  0.31138825
   -0.77083594]]

 ...

 [[ 0.32938877 -0.8027933   0.29306048 ... -0.78737634  0.32512185
   -0.79521805]
  [ 0.31824762 -0.78301376  0.29237252 ... -0.78068626  0.31539708
   -0.7779531 ]]

 [[ 0.32925186 -0.8025503   0.2873691  ... -0.7910959   0.32762644
   -0.79966456]
  [ 0.3209238  -0.78776497  0.28819722 ... -0.778897    0.30786255
   -0.7645766 ]]

 [[ 0.32515183 -0.7952712   0.29406002 ... -0.7854287   0.32059377
   -0.78717905]
  [ 0.3173912  -0.78149325  0.28922015 ... -0.77581364  0.3127626
   -0

[[   2   40 1191 ...    0    0    0]
 [   2    9   31 ...    0    0    0]
 [   2  176   41 ...    0    0    0]
 ...
 [   2   42   17 ...    0    0    0]
 [   2   72   31 ...    0    0    0]
 [   2   41  263 ...    0    0    0]]
25
[[[ 0.32582104 -0.7964594   0.29407534 ... -0.79670554  0.3317391
   -0.80696595]
  [ 0.31945878 -0.78516406  0.2865773  ... -0.78040874  0.31064886
   -0.7695233 ]]

 [[ 0.32978308 -0.8034933   0.29038683 ... -0.7857083   0.32137346
   -0.7885633 ]
  [ 0.32083896 -0.78761435  0.28569523 ... -0.7770178   0.30888802
   -0.7663971 ]]

 [[ 0.32697454 -0.79850715  0.28914285 ... -0.78696746  0.3268321
   -0.7982543 ]
  [ 0.320803   -0.7875505   0.28799945 ... -0.77498215  0.3118812
   -0.7717112 ]]

 ...

 [[ 0.32891822 -0.8019579   0.28690892 ... -0.78438747  0.32444343
   -0.79401356]
  [ 0.32018527 -0.78645384  0.29092786 ... -0.77814806  0.3131085
   -0.77388996]]

 [[ 0.32918924 -0.80243903  0.28861335 ... -0.78647685  0.32544258
   -0.79578745]
  [ 0.318022

[[  2  23 162 ...   0   0   0]
 [  2  29  68 ...   0   0   0]
 [  2  17 165 ...   0   0   0]
 ...
 [  2  15 138 ...   0   0   0]
 [  2  19  15 ...   0   0   0]
 [  2   9  67 ...   0   0   0]]
50
[[[ 0.32878882 -0.8017282   0.28791872 ... -0.78761506  0.32535005
   -0.7956232 ]
  [ 0.322528   -0.790613    0.28849235 ... -0.780059    0.3101007
   -0.76855016]]

 [[ 0.3286303  -0.80144674  0.29024673 ... -0.7817879   0.32789713
   -0.80014515]
  [ 0.32096583 -0.7878396   0.29149473 ... -0.7758682   0.3108136
   -0.7698158 ]]

 [[ 0.3278302  -0.8000263   0.2920514  ... -0.7912268   0.32830185
   -0.8008637 ]
  [ 0.31899375 -0.7843384   0.28818282 ... -0.7782342   0.31346166
   -0.774517  ]]

 ...

 [[ 0.3277366  -0.7998601   0.29237613 ... -0.7902669   0.32934466
   -0.80271494]
  [ 0.31670713 -0.7802789   0.28828093 ... -0.7801345   0.30961004
   -0.767679  ]]

 [[ 0.32766166 -0.7997271   0.29147246 ... -0.7864524   0.32846388
   -0.80115134]
  [ 0.3213279  -0.78848237  0.28652266 ... -0.

75
[[[ 0.32790774 -0.800164    0.2879943  ... -0.78543144  0.3287714
   -0.80169725]
  [ 0.32011405 -0.7863274   0.2890848  ... -0.77810615  0.30668038
   -0.76247776]]

 [[ 0.32901594 -0.8021314   0.29031053 ... -0.7815584   0.32641396
   -0.797512  ]
  [ 0.3210381  -0.78796786  0.29211357 ... -0.77771604  0.31420884
   -0.7758435 ]]

 [[ 0.3322534  -0.80787903  0.29070565 ... -0.7922843   0.32372668
   -0.79274106]
  [ 0.3180317  -0.78263044  0.28555766 ... -0.7751988   0.31219524
   -0.77226865]]

 ...

 [[ 0.32332855 -0.79203427  0.29964122 ... -0.7923965   0.327124
   -0.7987725 ]
  [ 0.32057932 -0.7871534   0.29262978 ... -0.7749905   0.3214064
   -0.7886217 ]]

 [[ 0.3306123  -0.8049655   0.28715715 ... -0.7857997   0.32758862
   -0.79959744]
  [ 0.3227489  -0.79100513  0.2889901  ... -0.77124536  0.3109844
   -0.770119  ]]

 [[ 0.32959315 -0.80315626  0.2889335  ... -0.7910619   0.32700655
   -0.7985641 ]
  [ 0.32136813 -0.7885538   0.28976053 ... -0.7780235   0.31224936
   -0.

# model designinig


In [146]:
num_layer = 4
d_model = 512
dff = 512
num_heads = 8
row_size = 2
col_size = 512

target_vocab_size = len(tokenizer.word_index) + 1
dropout_rate = 0.5

In [147]:
def get_angles(pos, i, d_model):
    angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
    #print("get angel")
    #print("angel shape",angle_rates.shape)
    return pos * angle_rates

In [148]:
def positional_encoding_2d(row,col,d_model):
    assert d_model % 2 == 0
    row_pos = np.repeat(np.arange(row),col)[:,np.newaxis]
    col_pos = np.repeat(np.expand_dims(np.arange(col),0),row,axis=0).reshape(-1,1)

    angle_rads_row = get_angles(row_pos,np.arange(d_model//2)[np.newaxis,:],d_model//2)
    angle_rads_col = get_angles(col_pos,np.arange(d_model//2)[np.newaxis,:],d_model//2)

    angle_rads_row[:, 0::2] = np.sin(angle_rads_row[:, 0::2])
    angle_rads_row[:, 1::2] = np.cos(angle_rads_row[:, 1::2])
    angle_rads_col[:, 0::2] = np.sin(angle_rads_col[:, 0::2])
    angle_rads_col[:, 1::2] = np.cos(angle_rads_col[:, 1::2])
    pos_encoding = np.concatenate([angle_rads_row,angle_rads_col],axis=1)[np.newaxis, ...]
    return tf.cast(pos_encoding, dtype=tf.float32)

In [149]:
def positional_encoding_1d(position, d_model):
    #print("positional encoding 1 dim")
    angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                          np.arange(d_model)[np.newaxis, :],
                          d_model)

    # apply sin to even indices in the array; 2i
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

    # apply cos to odd indices in the array; 2i+1
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

    pos_encoding = angle_rads[np.newaxis, ...]

    return tf.cast(pos_encoding, dtype=tf.float32)

In [150]:
def create_padding_mask(seq):
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)

    # add extra dimensions to add the padding
    # to the attention logits.
    return seq[:, tf.newaxis, tf.newaxis, :]  # (batch_size, 1, 1, seq_len)

In [151]:
def create_look_ahead_mask(size):
    mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
    return mask  # (seq_len, seq_len)

In [152]:
def scaled_dot_product_attention(q, k, v, mask):
    """Calculate the attention weights.
    q, k, v must have matching leading dimensions.
    k, v must have matching penultimate dimension, i.e.: seq_len_k = seq_len_v.
    The mask has different shapes depending on its type(padding or look ahead) 
    but it must be broadcastable for addition.

    Args:
    q: query shape == (..., seq_len_q, depth)
    k: key shape == (..., seq_len_k, depth)
    v: value shape == (..., seq_len_v, depth_v)
    mask: Float tensor with shape broadcastable 
          to (..., seq_len_q, seq_len_k). Defaults to None.

    Returns:
    output, attention_weights
    """

    matmul_qk = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)

    # scale matmul_qk
    dk = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

    # add the mask to the scaled tensor.
    if mask is not None:
        scaled_attention_logits += (mask * -1e9)  #adding -Inf where mask is 1 s.t. value get ignored in softmax

    # softmax is normalized on the last axis (seq_len_k) so that the scores
    # add up to 1.
    attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)  # (..., seq_len_q, seq_len_k)

    output = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)

    return output, attention_weights

In [153]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads

        self.wq = tf.keras.layers.Dense(d_model)
        self.wk = tf.keras.layers.Dense(d_model)
        self.wv = tf.keras.layers.Dense(d_model)

        self.dense = tf.keras.layers.Dense(d_model)

    def split_heads(self, x, batch_size):
        """Split the last dimension into (num_heads, depth).
        Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
        """
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, v, k, q, mask=None):
        #print("multihead attention")
        batch_size = tf.shape(q)[0]

        q = self.wq(q)  # (batch_size, seq_len, d_model)
        k = self.wk(k)  # (batch_size, seq_len, d_model)
        v = self.wv(v)  # (batch_size, seq_len, d_model)

        q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
        k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
        v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)

        # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
        # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
        scaled_attention, attention_weights = scaled_dot_product_attention(
            q, k, v, mask)

        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

        concat_attention = tf.reshape(scaled_attention, 
                                      (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)

        output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)

        return output, attention_weights

In [154]:
def point_wise_feed_forward_network(d_model, dff):
    return tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),  # (batch_size, seq_len, dff)
      tf.keras.layers.Dense(d_model)  # (batch_size, seq_len, d_model)
    ])

In [155]:
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.5):
        super(EncoderLayer, self).__init__()
        

        self.mha = MultiHeadAttention(d_model, num_heads)
        self.ffn = point_wise_feed_forward_network(d_model, dff)

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)

    def call(self, x, training, mask=None):
        #print("encoder layer chota")
        

        attn_output, _ = self.mha(x, x, x, mask)  # (batch_size, input_seq_len, d_model)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(x + attn_output)  # (batch_size, input_seq_len, d_model)

        ffn_output = self.ffn(out1)  # (batch_size, input_seq_len, d_model)
        ffn_output = self.dropout2(ffn_output, training=training)
        out2 = self.layernorm2(out1 + ffn_output)  # (batch_size, input_seq_len, d_model)

        return out2

In [156]:
class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.5):
        super(DecoderLayer, self).__init__()

        self.mha1 = MultiHeadAttention(d_model, num_heads)
        self.mha2 = MultiHeadAttention(d_model, num_heads)

        self.ffn = point_wise_feed_forward_network(d_model, dff)

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)
        self.dropout3 = tf.keras.layers.Dropout(rate)
    
    
    def call(self, x, enc_output, training, 
           look_ahead_mask=None, padding_mask=None):
        #print("decoder layer")
        # enc_output.shape == (batch_size, input_seq_len, d_model)

        # using look ahead mask so that during self attention current query dont consider future token
        attn1, attn_weights_block1 = self.mha1(x, x, x, look_ahead_mask)  # (batch_size, target_seq_len, d_model)
        attn1 = self.dropout1(attn1, training=training)
        out1 = self.layernorm1(attn1 + x)

        # use padding mask to avoid padded values of both enc_output and dec_input
        attn2, attn_weights_block2 = self.mha2(
            enc_output, enc_output, out1, padding_mask)  # (batch_size, target_seq_len, d_model)
        attn2 = self.dropout2(attn2, training=training)
        out2 = self.layernorm2(attn2 + out1)  # (batch_size, target_seq_len, d_model)

        ffn_output = self.ffn(out2)  # (batch_size, target_seq_len, d_model)
        ffn_output = self.dropout3(ffn_output, training=training)
        out3 = self.layernorm3(ffn_output + out2)  # (batch_size, target_seq_len, d_model)

        return out3, attn_weights_block1, attn_weights_block2

In [157]:
class Encoder(tf.keras.layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, dff,
               row_size,col_size,rate=0.5):
        super(Encoder, self).__init__()

        self.d_model = d_model
        self.num_layers = num_layers
        self.embedding = tf.keras.layers.Dense(self.d_model,activation='relu')
        #self.fnc = FeatureEmbedder(d_model,512)

        
        self.pos_encoding = positional_encoding_2d(row_size,col_size, 
                                                self.d_model)


        self.enc_layers = [EncoderLayer(d_model, num_heads, dff, rate) 
                           for _ in range(num_layers)]

        self.dropout = tf.keras.layers.Dropout(rate)

    def call(self, x, training, mask=None):
        # shape(x) = (batch_size,seq_len(H*W),features)
        #x = self.fnc(x)
        #print("encoder layer")
        #print("shape of x is ",x.shape)
        seq_len = tf.shape(x)[1]
        #print(tf.shape(x)[1])

        # adding embedding and position encoding.
        x = self.embedding(x)  # (batch_size, input_seq_len(H*W), d_model)
        x += self.pos_encoding[:, :seq_len]

        x = self.dropout(x, training=training)

        for i in range(self.num_layers):
            x = self.enc_layers[i](x, training, mask)

        return x  # (batch_size, input_seq_len, d_model)

In [158]:
class Decoder(tf.keras.layers.Layer):
    def __init__(self, num_layers,d_model,num_heads,dff, target_vocab_size, maximum_position_encoding,   rate=0.5):
        super(Decoder, self).__init__()
        self.d_model = d_model
        self.num_layers = num_layers

        self.embedding = tf.keras.layers.Embedding(target_vocab_size, d_model)
        self.pos_encoding = positional_encoding_1d(maximum_position_encoding, d_model)

        self.dec_layers = [DecoderLayer(d_model, num_heads, dff, rate)
                         for _ in range(num_layers)]
        self.dropout = tf.keras.layers.Dropout(rate)

    def call(self, x, enc_output, training,look_ahead_mask=None, padding_mask=None):
        seq_len = tf.shape(x)[1]
        attention_weights = {}
        #print("shape of seq_len ",seq_len)
        #print("x shape in decoder cakk",x.shape)

        x = self.embedding(x)  # (batch_size, target_seq_len, d_model)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x += self.pos_encoding[:, :seq_len, :]
        x = self.dropout(x, training=training)

        for i in range(self.num_layers):
            x, block1, block2 = self.dec_layers[i](x, enc_output, training,
                                            look_ahead_mask, padding_mask)

            attention_weights['decoder_layer{}_block1'.format(i+1)] = block1
            attention_weights['decoder_layer{}_block2'.format(i+1)] = block2

        return x, attention_weights

In [159]:
class Transformer(tf.keras.Model):
    def __init__(self, num_layers, d_model, num_heads, dff,row_size,col_size,
              target_vocab_size,max_pos_encoding, rate=0.5):
        super(Transformer, self).__init__()

        self.encoder = Encoder(num_layers, d_model, num_heads, dff,row_size,col_size, rate)
        self.decoder = Decoder(num_layers, d_model, num_heads, dff,
                          target_vocab_size,max_pos_encoding, rate)
        self.final_layer = tf.keras.layers.Dense(target_vocab_size)

    def call(self, inp, tar, training,look_ahead_mask=None,dec_padding_mask=None,enc_padding_mask=None ):
        # print("inp.shape",inp.shape)
        # print("training shape",training.shape)
        # print("enc_padding_mask ",enc_padding_mask.shape)
        enc_output = self.encoder(inp, training)  # (batch_size, inp_seq_len, d_model      )
        dec_output, attention_weights = self.decoder(
        tar, enc_output, training, look_ahead_mask, dec_padding_mask)
        final_output = self.final_layer(dec_output)  # (batch_size, tar_seq_len, target_vocab_size)
        return final_output, attention_weights

In [160]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()

        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps

    def __call__(self, step):
        #print("custom schedule")
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [161]:
learning_rate = CustomSchedule(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.999, 
                                     epsilon=1e-9)

In [162]:

loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

In [163]:

def loss_function(real, pred):
    #print("loss_function")
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_sum(loss_)/tf.reduce_sum(mask)

In [164]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(
    name='train_accuracy')

In [165]:
transformer = Transformer(num_layer,d_model,num_heads,dff,row_size,col_size,target_vocab_size,max_pos_encoding=target_vocab_size,rate=dropout_rate)

In [166]:

def create_masks_decoder(tar):
    #print("msak decoder run")
    look_ahead_mask = create_look_ahead_mask(tf.shape(tar)[1])
    dec_target_padding_mask = create_padding_mask(tar)
    combined_mask = tf.maximum(dec_target_padding_mask, look_ahead_mask)
    return combined_mask

In [167]:
@tf.function
def train_step(img_tensor, tar):
   #print("train run")
    tar_inp = tar[:, :-1]
    #print(tar_inp)

    tar_real = tar[:, 1:]
    # print(tar_real)
    dec_mask = create_masks_decoder(tar_inp)
    #print("j9j")
    with tf.GradientTape() as tape:
        predictions, _ = transformer(img_tensor, tar_inp,True, dec_mask)
        loss = loss_function(tar_real, predictions)

    gradients = tape.gradient(loss, transformer.trainable_variables)   
    optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))
    train_loss(loss)
    train_accuracy(tar_real, predictions)

In [ ]:
for epoch in range(300):
    start = time.time()

    train_loss.reset_states()
    train_accuracy.reset_states()
  
    for (batch, (img_tensor, tar)) in enumerate(dataset.as_numpy_iterator()):
        #print("eopch run")
        train_step(img_tensor, tar)

        if batch % 50 == 0:
            

            print ('Epoch {} Batch {} Loss {:.4f} Accuracy {:.4f}'.format(
              epoch + 1, batch, train_loss.result(), train_accuracy.result()))


    print ('Epoch {} Loss {:.4f} Accuracy {:.4f}'.format(epoch + 1, 
                                                train_loss.result(), 
                                                train_accuracy.result()))

    print ('Time taken for 1 epoch: {} secs\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 8.2139 Accuracy 0.0005
Epoch 1 Batch 50 Loss 8.1207 Accuracy 0.0016
Epoch 1 Batch 100 Loss 7.8487 Accuracy 0.0139
Epoch 1 Loss 7.7827 Accuracy 0.0153
Time taken for 1 epoch: 39.962167739868164 secs

Epoch 2 Batch 0 Loss 7.0399 Accuracy 0.0294
Epoch 2 Batch 50 Loss 6.6748 Accuracy 0.0294
Epoch 2 Batch 100 Loss 6.3291 Accuracy 0.0294
Epoch 2 Loss 6.2614 Accuracy 0.0294
Time taken for 1 epoch: 23.395339488983154 secs

Epoch 3 Batch 0 Loss 5.5626 Accuracy 0.0294
Epoch 3 Batch 50 Loss 5.4018 Accuracy 0.0294
Epoch 3 Batch 100 Loss 5.2955 Accuracy 0.0294
Epoch 3 Loss 5.2809 Accuracy 0.0294
Time taken for 1 epoch: 23.549812078475952 secs

Epoch 4 Batch 0 Loss 5.0756 Accuracy 0.0294
Epoch 4 Batch 50 Loss 5.1149 Accuracy 0.0294
Epoch 4 Batch 100 Loss 5.0931 Accuracy 0.0294
Epoch 4 Loss 5.0880 Accuracy 0.0294
Time taken for 1 epoch: 23.58858633041382 secs

Epoch 5 Batch 0 Loss 4.9970 Accuracy 0.0294
Epoch 5 Batch 50 Loss 5.0726 Accuracy 0.0294
Epoch 5 Batch 100 Loss 5.0581 Ac